# \[STBDA2023\] 03wk-013: 타이타닉, 로지스틱

김보람  
2023-09-28

> 해당 자료는 전북대학교 최규빈 교수님 2023학년도 2학기
> [빅데이터분석특강](https://guebin.github.io/STBDA2023/) 자료임

# 03wk-013: 타이타닉, 로지스틱

최규빈  
2023-09-21

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-wi9Mkfc849jTCMENvydI8B&si=hotGT-ErLB8dukhs>

# 2. Import

In [1]:
import numpy as np
import pandas as pd 
import sklearn.linear_model

# 3. Data 불러오기

In [3]:
df_train = pd.read_csv('~/Desktop/titanic/train.csv')
df_test = pd.read_csv('~/Desktop/titanic/test.csv')

In [4]:
df_train

In [5]:
set(df_train) - set(df_test)

train에는 있지만 test에는 없는 것 Survived -\> y로 놓자

# 4. 분석 – 실패

## A. 데이터 정리

In [11]:
X = pd.get_dummies(df_train.drop(['PassengerId','Survived'],axis=1))
y = df_train[['Survived']]

## B. Predictor 생성

In [12]:
predictr = sklearn.linear_model.LogisticRegression()
predictr 

## C. 학습 (fit, learn)

In [13]:
predictr.fit(X,y)

-   X가 NaN값이 있어서 오류남

# 5. 원인분석

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

`-` 문제1: `Cabin`열은 너무 많은 결측치를 가지고 있다. (추후에 빼자.)

`-` 문제2: `Name` 혹은 `Ticket`과 같은 변수는 one-hot 인코딩 하기
어색하다. (object 변수)

In [15]:
len(set(df_train['Name']))

In [16]:
len(set(df_train['Ticket']))

`-` 문제3: df_train의 `Age`와 `Embarked`에 약간 포함된 결측치가 마음에
걸린다.. $\to$ 빼자!

`-` 문제4: df_test의 `Fare`에 포함된 결측값도 걸린다. $\to$ 빼자!

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB

-   Fare는 417인데 애매하구먼? autogluon에서는 어떻게 했었나 한번
    살펴보자

# 6. 분석 – 성공

## A. 데이터정리

In [37]:
X = pd.get_dummies(df_train[["Pclass", "Sex", "SibSp", "Parch"]])
y = df_train[["Survived"]]

In [38]:
X

## B. Predictor 생성

In [20]:
predictr = sklearn.linear_model.LogisticRegression()

## C. 학습

In [21]:
predictr.fit(X, y)

/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

## D. 예측

In [22]:
#predictr.predict(X)
df_train.assign(Survived_hat=predictr.predict(X)).loc[:,['Survived','Survived_hat']]

## E. 평가

In [23]:
predictr.score(X,y)

# 7. 제출 (HW)

In [26]:
df_test

In [33]:
X = pd.get_dummies(df_test[["Pclass", "Sex", "SibSp", "Parch"]])

In [36]:
X

In [35]:
df_test.assign(Survived=predictr.predict(X)).loc[:,['PassengerId','Survived']]\
.to_csv("03wk(get_dummies).csv",index=False)

`-` kaggle 제출 -\> 0.77511